# Лабораторная работа №6


In [ ]:
import numpy as np
from scipy import stats
from IPython.display import display, Latex
from sklearn.linear_model import HuberRegressor

# Установка seed для воспроизводимости
np.random.seed(42)

n = 500

alpha = 0.05

## 1. Критерий Хсу


### 1.1. $X_n \in N(5, 10)$


In [ ]:
x = np.random.normal(5, 10, n)
H = (np.sum((np.arange(n)) * np.pow(x - np.median(x), 2))) / (
    (n - 1) * np.sum(np.pow(x - np.median(x), 2))
)
DH = (n + 1) / (6 * (n - 1) * (n + 2))
H_standard = (H - 0.5) / (np.sqrt(DH))
q = stats.t.ppf(1 - alpha / 2, df=n - 1)

display(Latex(f"$H_{{standard}}={H_standard}$"))

if np.abs(H_standard) < q:
    display(Latex("$H_0$ accepted"))
else:
    display(Latex("$H_0$ declined"))

### 1.2. a) $X_{\frac{n}{2}} \in N(5, 10)$ + $X_{\frac{n}{2}} \in N(5, 11)$


In [ ]:
x = np.hstack([np.random.normal(5, 10, n // 2), np.random.normal(5, 11, n // 2)])

H = (np.sum((np.arange(n)) * np.pow(x - np.median(x), 2))) / (
    (n - 1) * np.sum(np.pow(x - np.median(x), 2))
)
DH = (n + 1) / (6 * (n - 1) * (n + 2))
H_standard = (H - 0.5) / (np.sqrt(DH))
q = stats.t.ppf(1 - alpha / 2, df=n - 1)

display(Latex(f"$H_{{standard}}={H_standard}$"))

if np.abs(H_standard) < q:
    display(Latex("$H_0$ accepted"))
else:
    display(Latex("$H_0$ declined"))

### 1.2. б) $X_{\frac{n}{2}} \in N(5, 10)$ + $X_{\frac{n}{2}} \in N(5, 1)$


In [ ]:
x = np.hstack([np.random.normal(5, 10, n // 2), np.random.normal(5, 1, n // 2)])

H = (np.sum((np.arange(n)) * np.pow(x - np.median(x), 2))) / (
    (n - 1) * np.sum(np.pow(x - np.median(x), 2))
)
DH = (n + 1) / (6 * (n - 1) * (n + 2))
H_standard = (H - 0.5) / (np.sqrt(DH))
q = stats.t.ppf(1 - alpha / 2, df=n - 1)

display(Latex(f"$H_{{standard}}={H_standard}$"))

if np.abs(H_standard) < q:
    display(Latex("$H_0$ accepted"))
else:
    display(Latex("$H_0$ declined"))

## 2. Выбросы


### 2.1. Три сигмы


In [ ]:
x = np.hstack([np.random.normal(0, 1, 195), [-3, -4, 2.99, 3.3, 5]])

s = x.std()

for i in range(len(x)):
    if not (-3 * s <= x[i] <= 3 * s):
        display(Latex(f"$i = {i+1}$, $x_i = {x[i]}$"))

### 2.2. Боксплот Тьюки


In [ ]:
x = np.hstack([np.random.normal(0, 1, 195), [-3, -4, 2.99, 3.3, 5]])

lq = np.quantile(x, 0.25)
uq = np.quantile(x, 0.75)
iq = uq - lq
xu = uq + 3/2 * iq
xl = lq - 3/2 * iq

for i in range(len(x)):
    if not (xl <= x[i] <= xu):
        display(Latex(f"$i = {i+1}$, $x_i = {x[i]}$"))

## 3. Сравнение робастных и неробастных оценок

In [ ]:
def huber_estimator(x, delta=1.0, tol=1e-6, max_iter=100):
    theta = np.median(x)
    
    for _ in range(max_iter):
        # Вычисление остатков
        residuals = x - theta
        
        # Весовая функция Хубера
        weights = np.where(
            np.abs(residuals) <= delta,
            1.0,
            delta / np.abs(residuals)
        )
        
        # Обновление оценки как взвешенного среднего
        theta_new = np.sum(weights * x) / np.sum(weights)
        
        # Проверка на сходимость
        if np.abs(theta_new - theta) < tol:
            break
            
        theta = theta_new
    
    return theta


In [ ]:
def compare_estimators(data):
    mean = np.mean(data)
    median = np.median(data)
    huber = huber_estimator(data, delta=5)
    return {"Среднее": mean, "Медиана": median, "Huber": huber}



### 3.1. $X_n \in N(0, 1)$

In [ ]:
x = np.random.normal(0, 1, n)
display(Latex("$" + "\\newline ".join([f"{c[0]}: {c[1]}" for c in compare_estimators(x).items()]) + "$"))

### 3.2. $X_n \in Cauchy$

In [ ]:
x = np.random.standard_cauchy(n)
display(Latex("$" + "\\newline ".join([f"{c[0]}: {c[1]}" for c in compare_estimators(x).items()]) + "$"))


### 3.3. $X_n \in N(0, 1) + 0.1\cdot Cauchy$

In [ ]:
x = 0.1*np.random.standard_cauchy(n) + np.random.normal(0, 1, n)
display(Latex("$" + "\\newline ".join([f"{c[0]}: {c[1]}" for c in compare_estimators(x).items()]) + "$"))
